In [1]:
import cv2
import mediapipe as mp #face detector
import math
import numpy as np
import time
import datetime
from collections import Counter
import onnxruntime as ort
from pathlib import Path

import warnings
warnings.simplefilter("ignore", UserWarning)

import torch
import torch.nn as  nn
import torch.nn.functional as F
from PIL import Image
from torchvision import transforms

# Import tkinter for GUI
import tkinter as tk
from tkinter import ttk, scrolledtext, filedialog
import PIL.Image, PIL.ImageTk
from threading import Thread, Event
import os

# Additional imports for audio processing
import pyaudio
import wave
import threading
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
import queue
import soundfile as sf
import librosa

# Import transformers for audio processing
from transformers import pipeline

# Set up ONNX Runtime for Hexagon
ONNXRUNTIME_DIR = Path(ort.__file__).parent

HEXAGON_DRIVER = Path.joinpath(ONNXRUNTIME_DIR, "capi", "QnnHtp.dll")

QNN_PROVIDER_OPTIONS = {"backend_path": HEXAGON_DRIVER,}

SESS_OPTS = ort.SessionOptions()

import subprocess

c:\Users\HAQKATHON SCL\haQathon\myenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Model architectures

#### Sub functions

In [2]:
def pth_processing_np(fp):
    """
    Numpy version of pth_processing function
    Performs the same preprocessing as the PyTorch version but using numpy
    
    Args:
        fp: PIL Image or numpy array
    
    Returns:
        numpy array with shape (1, 3, 224, 224) ready for model input
    """
    def preprocess_input_np(img_array):
        """Apply the same preprocessing as PyTorch version"""
        # Convert to float32
        x = img_array.astype(np.float32)
        
        # Flip channels (RGB -> BGR, equivalent to torch.flip(dims=(0,)))
        x = x[:, :, ::-1]
        
        # Apply normalization (same values as PyTorch version)
        x[:, :, 0] -= 91.4953   # Blue channel
        x[:, :, 1] -= 103.8827  # Green channel  
        x[:, :, 2] -= 131.0912  # Red channel
        
        return x
    
    def get_img_numpy(img):
        """Convert PIL image to preprocessed numpy array"""
        # Resize image to 224x224 (same as PyTorch version)
        img = img.resize((224, 224), Image.Resampling.NEAREST)
        
        # Convert PIL to numpy array
        img_array = np.array(img)
        
        # Apply preprocessing
        img_array = preprocess_input_np(img_array)
        
        # Transpose from HWC to CHW format (Height, Width, Channels -> Channels, Height, Width)
        img_array = np.transpose(img_array, (2, 0, 1))
        
        # Add batch dimension (equivalent to torch.unsqueeze(img, 0))
        img_array = np.expand_dims(img_array, axis=0)
        
        return img_array
    
    return get_img_numpy(fp)

def norm_coordinates(normalized_x, normalized_y, image_width, image_height):
    
    x_px = min(math.floor(normalized_x * image_width), image_width - 1)
    y_px = min(math.floor(normalized_y * image_height), image_height - 1)
    
    return x_px, y_px

def get_box(fl, w, h):
    idx_to_coors = {}
    for idx, landmark in enumerate(fl.landmark):
        landmark_px = norm_coordinates(landmark.x, landmark.y, w, h)

        if landmark_px:
            idx_to_coors[idx] = landmark_px

    x_min = np.min(np.asarray(list(idx_to_coors.values()))[:,0])
    y_min = np.min(np.asarray(list(idx_to_coors.values()))[:,1])
    endX = np.max(np.asarray(list(idx_to_coors.values()))[:,0])
    endY = np.max(np.asarray(list(idx_to_coors.values()))[:,1])

    (startX, startY) = (max(0, x_min), max(0, y_min))
    (endX, endY) = (min(w - 1, endX), min(h - 1, endY))
    
    return startX, startY, endX, endY

In [3]:
class AudioVisualizer:
    def __init__(self, frame, height=100, width=200):
        self.frame = frame
        self.height = height
        self.width = width
        
        # Create matplotlib figure
        self.fig = Figure(figsize=(width/100, height/100), dpi=100)
        self.ax = self.fig.add_subplot(111)
        
        # Initial empty plot
        self.line, = self.ax.plot([], [], color='#3498db', linewidth=2)
        self.ax.set_ylim(-0.5, 0.5)
        self.ax.set_xlim(0, 100)
        self.ax.axis('off')
        self.fig.subplots_adjust(left=0, right=1, top=1, bottom=0, wspace=0, hspace=0)
        
        # Create canvas
        self.canvas = FigureCanvasTkAgg(self.fig, master=self.frame)
        self.canvas.get_tk_widget().pack(fill=tk.BOTH, expand=True)
        
        # Buffer for audio data
        self.buffer = [0] * 100
    
    def update(self, audio_data):
        # Convert audio data to normalized values
        if audio_data is not None and len(audio_data) > 0:
            # Ensure we're getting the right number of samples
            samples = np.frombuffer(audio_data, dtype=np.int16).astype(np.float32)
            samples = samples / 32768.0  # Normalize
            
            # Update buffer (sliding window)
            self.buffer = self.buffer[len(samples):] + samples.tolist()[:100]
            
            # Update plot
            self.line.set_data(range(len(self.buffer)), self.buffer)
            self.canvas.draw_idle()

class AudioProcessor:
    def __init__(self, visualizer=None, callback=None):
        self.CHUNK = 1024
        self.FORMAT = pyaudio.paInt16
        self.CHANNELS = 1
        self.RATE = 16000  # 16kHz for model
        self.recording = False
        self.frames = []
        self.visualizer = visualizer
        self.callback = callback
        self.audio_queue = queue.Queue()
        self.audio = pyaudio.PyAudio()
        self.stream = None
        
        # Speech characteristics
        self.speech_metrics = {
            "avg_pitch": 0,
            "pitch_variation": 0,
            "speaking_rate": 0,
            "volume": 0,
            "clarity": 0,
            "samples_processed": 0
        }
        
        # Voice emotions (derived from metrics only)
        self.voice_emotions = {
            "confidence": 0,
            "engagement": 0,
            "hesitation": 0,
            "enthusiasm": 0
        }
        
        # Flag for periodic processing
        self.last_process_time = 0
        self.processing_interval = 5  # Process every 5 seconds

    def start_recording(self):
        if self.recording:
            return
        self.recording = True
        self.frames = []
        self.last_process_time = time.time()
        
        # Reset metrics
        self.speech_metrics = {
            "avg_pitch": 0,
            "pitch_variation": 0,
            "speaking_rate": 0,
            "volume": 0,
            "clarity": 0,
            "samples_processed": 0
        }
        
        self.voice_emotions = {
            "confidence": 0,
            "engagement": 0,
            "hesitation": 0,
            "enthusiasm": 0
        }
        
        def callback(in_data, frame_count, time_info, status):
            if self.recording:
                self.frames.append(in_data)
                self.audio_queue.put(in_data)
                
                # Update visualizer if available
                if self.visualizer:
                    self.visualizer.update(in_data)
                
                # Process audio periodically
                current_time = time.time()
                if current_time - self.last_process_time >= self.processing_interval:
                    self.process_audio_chunk()
                    self.last_process_time = current_time
            return (in_data, pyaudio.paContinue)
        
        try:
            self.stream = self.audio.open(
                format=self.FORMAT,
                channels=self.CHANNELS,
                rate=self.RATE,
                input=True,
                frames_per_buffer=self.CHUNK,
                stream_callback=callback
            )
            print("Audio recording started")
        except Exception as e:
            print(f"Error starting audio stream: {e}")
            self.recording = False

    def stop_recording(self):
        if not self.recording:
            return
        self.recording = False
        
        if self.stream:
            try:
                self.stream.stop_stream()
                self.stream.close()
                self.stream = None
                print("Audio recording stopped")
            except Exception as e:
                print(f"Error stopping audio stream: {e}")

    def process_audio_chunk(self):
        # Always process/log, even if not enough frames
        enough_data = len(self.frames) >= 50
        # Take last ~3 seconds of audio, or whatever is available
        recent_frames = self.frames[-50:] if enough_data else self.frames
        try:
            if recent_frames:
                audio_data = np.frombuffer(b''.join(recent_frames), dtype=np.int16).astype(np.float32)
                audio_data = audio_data / 32768.0  # Normalize to [-1, 1]
            else:
                audio_data = np.array([])

            # Only use direct speech metrics (no HuggingFace model)
            direct_metrics_emotions = {
                "confidence": 0,
                "engagement": 0,
                "hesitation": 0,
                "enthusiasm": 0
            }

            if len(audio_data) > 0:
                # Calculate volume (RMS amplitude)
                volume = np.sqrt(np.mean(audio_data**2))
                # In AudioProcessor.process_audio_chunk, after volume calculation
                if volume > 0:
                    volume_db = 20 * np.log10(volume)
                else:
                    volume_db = -100.0
                self.speech_metrics["volume_db"] = volume_db
                try:
                    # Extract pitch information
                    pitches, magnitudes = librosa.piptrack(y=audio_data, sr=self.RATE)
                    pitch_values = []
                    for i in range(pitches.shape[1]):
                        index = magnitudes[:, i].argmax()
                        pitch = pitches[index, i]
                        if pitch > 0:
                            pitch_values.append(pitch)
                    
                    if pitch_values:
                        # Calculate basic speech metrics
                        avg_pitch = np.mean(pitch_values)
                        pitch_variation = np.std(pitch_values)
                        alpha = 0.7  # Smoothing factor
                        
                        # Update speech metrics with new values
                        self.speech_metrics["avg_pitch"] = alpha * avg_pitch + (1 - alpha) * self.speech_metrics["avg_pitch"]
                        self.speech_metrics["pitch_variation"] = alpha * pitch_variation + (1 - alpha) * self.speech_metrics["pitch_variation"]
                        self.speech_metrics["volume"] = alpha * volume + (1 - alpha) * self.speech_metrics["volume"]
                        self.speech_metrics["samples_processed"] += 1
                        
                        # Calculate speaking rate from zero crossings
                        zero_crossings = librosa.zero_crossings(audio_data)
                        speaking_rate = sum(zero_crossings) / len(audio_data) * self.RATE / 100
                        self.speech_metrics["speaking_rate"] = alpha * speaking_rate + (1 - alpha) * self.speech_metrics["speaking_rate"]
                        
                        # Clarity (spectral centroid)
                        spectral_centroids = librosa.feature.spectral_centroid(y=audio_data, sr=self.RATE)[0]
                        self.speech_metrics["clarity"] = min(1.0, np.mean(spectral_centroids) / 3000)
                        
                        # Calculate emotions from direct speech metrics
                        # 1. Confidence: Based on volume and steady pitch
                        norm_pitch_var = min(1.0, pitch_variation / 200)  # Cap at 200Hz variation
                        steady_factor = 1.0 - norm_pitch_var * 0.5  # Steadier voice = more confident (but with less impact)
                        volume_factor = min(1.0, volume * 12)  # Louder = more confident, up to a limit
                        direct_metrics_emotions["confidence"] = 0.8 * volume_factor + 0.2 * steady_factor
                        
                        # 2. Enthusiasm: Based on pitch variation and speaking rate
                        norm_speaking_rate = min(1.0, speaking_rate / 15)  # Good rate is around 10-15
                        direct_metrics_emotions["enthusiasm"] = 0.6 * norm_pitch_var + 0.4 * norm_speaking_rate
                        
                        # 3. Hesitation: Inverse of speaking rate and confidence
                        slow_factor = 1.0 - min(1.0, speaking_rate / 8)  # Slower = more hesitation
                        direct_metrics_emotions["hesitation"] = 0.6 * slow_factor + 0.4 * (1.0 - volume_factor)
                        
                        # 4. Engagement: Based on clarity, pitch variation, and balanced speaking rate
                        rate_balance = 1.0 - abs(norm_speaking_rate - 0.5) * 2  # Penalize too fast/slow
                        direct_metrics_emotions["engagement"] = 0.4 * self.speech_metrics["clarity"] + 0.3 * norm_pitch_var + 0.3 * rate_balance
                        
                except Exception as e:
                    print(f"Error in audio metrics: {e}")

            # Use only direct metrics for emotions
            for emotion in self.voice_emotions:
                self.voice_emotions[emotion] = min(1.0, max(0.0, direct_metrics_emotions[emotion]))

            # Notify callback if available (always log an interval)
            if self.callback:
                self.callback(self.speech_metrics, self.voice_emotions)
        except Exception as e:
            print(f"Error processing audio chunk: {e}")

#### Emotion Logging GUI Application

In [4]:
class EmotionLoggingApp:
    def __init__(self, window, window_title):
        self.window = window
        self.window.title(window_title)
        self.window.geometry("1200x800")  # Increased height for summary
        
        # Initialize models
        self.init_models()
        
        # Create the UI elements
        self.create_ui()
        
        # Initialize variables
        self.is_running = False
        self.stop_event = Event()
        self.emotion_logs = []
        self.current_emotions = []
        self.speech_logs = []
        self.logging_start_time = None
        self.last_log_time = None
        self.transcript_file_path = None
        self.stopwatch_active = False
        self.elapsed_time = 0
        
        # Start video capture
        self.cap = cv2.VideoCapture(0)
        self.update()
        
        self.window.protocol("WM_DELETE_WINDOW", self.on_closing)
        self.window.mainloop()
    
    def init_models(self):
        # MediaPipe setup
        self.mp_face_mesh = mp.solutions.face_mesh
        
        name_LSTM_model = 'lstm_model.onnx'
        name_resnet_feature_model = 'resnet_feature_extractor.onnx'
        
        self.session_lstm = ort.InferenceSession(name_LSTM_model, 
                                            providers=[("QNNExecutionProvider", QNN_PROVIDER_OPTIONS),
                                                       "CPUExecutionProvider"], sess_options=SESS_OPTS)
        
        self.session_feature = ort.InferenceSession(name_resnet_feature_model,
                                               providers=[("QNNExecutionProvider", QNN_PROVIDER_OPTIONS),
                                                          "CPUExecutionProvider"], sess_options=SESS_OPTS)
        self.lstm_input_name =  self.session_lstm.get_inputs()[0].name
        self.feature_extractor_input_name =  self.session_feature.get_inputs()[0].name
        # Load ResNet model
        print('Facial feature extraction model loaded successfully.')
        # Emotion dictionary
        self.DICT_EMO = {0: 'Neutral', 1: 'Happiness', 2: 'Sadness', 3: 'Surprise', 4: 'Fear', 5: 'Disgust', 6: 'Anger'}
        self.NEGATIVE_EMOTIONS = ['Sadness', 'Fear', 'Disgust', 'Anger']
        
        # Initialize LSTM features
        self.lstm_features = []
    
    def create_ui(self):
        # Main frame
        main_frame = ttk.Frame(self.window)
        main_frame.pack(fill=tk.BOTH, expand=True, padx=10, pady=10)
        
        # Left frame for video and transcript
        left_frame = ttk.Frame(main_frame)
        left_frame.pack(side=tk.LEFT, fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Transcript frame at the top of left frame
        transcript_frame = ttk.LabelFrame(left_frame, text="Presentation Transcript")
        transcript_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Transcript buttons frame
        transcript_buttons_frame = ttk.Frame(transcript_frame)
        transcript_buttons_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Choose file button
        self.choose_file_button = ttk.Button(transcript_buttons_frame, text="Choose Transcript File", command=self.choose_transcript_file)
        self.choose_file_button.pack(side=tk.LEFT, padx=5)
        
        # File name label
        self.file_label = ttk.Label(transcript_buttons_frame, text="No file selected")
        self.file_label.pack(side=tk.LEFT, padx=5)
        
        # Transcript text area
        self.transcript_text = scrolledtext.ScrolledText(transcript_frame, wrap=tk.WORD, height=8)
        self.transcript_text.pack(fill=tk.X, padx=5, pady=5)
        self.transcript_text.insert(tk.END, "Load a transcript file to view your presentation text here.")
        
        # Video frame
        self.video_frame = ttk.LabelFrame(left_frame, text="Feed: ")
        self.video_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Create canvas for video
        self.canvas = tk.Canvas(self.video_frame)
        self.canvas.pack(fill=tk.BOTH, expand=True)
        
        # Audio visualizer frame below video
        audio_viz_frame = ttk.LabelFrame(left_frame, text="Audio Input Level")
        audio_viz_frame.pack(fill=tk.X, padx=5, pady=5)
        viz_content_frame = ttk.Frame(audio_viz_frame, height=120)
        viz_content_frame.pack(fill=tk.X)
        viz_content_frame.pack_propagate(False)
        self.audio_visualizer = AudioVisualizer(viz_content_frame, height=100, width=800)
        
        # Right frame for controls and summary
        right_frame = ttk.Frame(main_frame, width=300)
        right_frame.pack(side=tk.RIGHT, fill=tk.BOTH, padx=5, pady=5, expand=False)
        right_frame.pack_propagate(False)
        
        # Control frame
        control_frame = ttk.LabelFrame(right_frame, text="Controls")
        control_frame.pack(fill=tk.X, padx=5, pady=5)
        
        # Stopwatch display
        stopwatch_frame = ttk.Frame(control_frame)
        stopwatch_frame.pack(fill=tk.X, padx=5, pady=5)
        
        ttk.Label(stopwatch_frame, text="Elapsed Time: ").pack(side=tk.LEFT)
        self.time_label = ttk.Label(stopwatch_frame, text="00:00:00", font=("Arial", 14, "bold"))
        self.time_label.pack(side=tk.LEFT, padx=5)
        
        # Start button
        self.start_button = ttk.Button(control_frame, text="Start", command=self.start_logging)
        self.start_button.pack(fill=tk.X, padx=5, pady=5)
        
        # Stop button
        self.stop_button = ttk.Button(control_frame, text="Stop", command=self.stop_logging, state=tk.DISABLED)
        self.stop_button.pack(fill=tk.X, padx=5, pady=5)
        
        # Reset button
        self.reset_button = ttk.Button(control_frame, text="Reset", command=self.reset_logging, state=tk.DISABLED)
        self.reset_button.pack(fill=tk.X, padx=5, pady=5)
        
        # Status indicator
        status_frame = ttk.Frame(control_frame)
        status_frame.pack(fill=tk.X, padx=5, pady=5)
        
        ttk.Label(status_frame, text="Status: ").pack(side=tk.LEFT)
        self.status_label = ttk.Label(status_frame, text="Ready")
        self.status_label.pack(side=tk.LEFT)
        
        # --- Removed Voice Analysis section here ---
        
        # Add Analyze button above the summary frame
        self.analyze_button = ttk.Button(right_frame, text="Analyze", command=self.display_summary, state=tk.DISABLED)
        self.analyze_button.pack(fill=tk.X, padx=5, pady=5)
        
        # Summary frame (now taller)
        summary_frame = ttk.LabelFrame(right_frame, text="Analysis Summary")
        summary_frame.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        
        # Create scrolled text for summary (increased height)
        self.summary_text = scrolledtext.ScrolledText(summary_frame, wrap=tk.WORD, width=30, height=40)
        self.summary_text.pack(fill=tk.BOTH, expand=True, padx=5, pady=5)
        self.summary_text.config(state=tk.DISABLED)
        
        # Initialize audio processor (no update_voice_metrics callback)
        # self.audio_processor = AudioProcessor(self.audio_visualizer, None)
        # Initialize audio processor (now with update_voice_metrics callback)
        self.audio_processor = AudioProcessor(self.audio_visualizer, self.update_voice_metrics)
    
    def update_voice_metrics(self, metrics, emotions):
        """Callback to log audio metrics/emotions every interval."""
        # Only log if logging is active
        if self.is_running:
            log_entry = {
                "timestamp": int(time.time() - self.logging_start_time) if self.logging_start_time else 0,
                "metrics": metrics.copy(),
                "emotions": emotions.copy()
            }
            self.speech_logs.append(log_entry)
    
    # --- Removed update_voice_metrics method ---
    
    def choose_transcript_file(self):
        file_path = filedialog.askopenfilename(
            title="Select Transcript File",
            filetypes=[("Text files", "*.txt"), ("All files", "*.*")]
        )
        
        if file_path:
            self.transcript_file_path = file_path
            filename = os.path.basename(file_path)
            self.file_label.config(text=f"Selected: {filename}")
            
            try:
                with open(file_path, 'r') as file:
                    content = file.read()
                    self.transcript_text.delete(1.0, tk.END)
                    self.transcript_text.insert(tk.END, content)
            except Exception as e:
                self.transcript_text.delete(1.0, tk.END)
                self.transcript_text.insert(tk.END, f"Error reading file: {str(e)}")
    
    def update_stopwatch(self):
        if self.stopwatch_active:
            current_time = time.time()
            self.elapsed_time = current_time - self.logging_start_time
            
            # Format time as HH:MM:SS
            hours, remainder = divmod(int(self.elapsed_time), 3600)
            minutes, seconds = divmod(remainder, 60)
            time_str = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
            self.time_label.config(text=time_str)
            
            # Update every 1 second
            self.window.after(1000, self.update_stopwatch)
    
    def update(self):
        ret, frame = self.cap.read()
        
        if ret:
            # Process frame if logging is active
            if self.is_running:
                self.process_frame(frame)
            
            # Convert to RGB for display
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            img = PIL.Image.fromarray(frame_rgb)
            
            # Resize to fit canvas
            canvas_width = self.canvas.winfo_width()
            canvas_height = self.canvas.winfo_height()
            
            if canvas_width > 1 and canvas_height > 1:
                ratio = min(canvas_width/img.width, canvas_height/img.height)
                new_width = int(img.width * ratio)
                new_height = int(img.height * ratio)
                img = img.resize((new_width, new_height), PIL.Image.Resampling.LANCZOS)
            
            self.photo = PIL.ImageTk.PhotoImage(image=img)
            self.canvas.create_image(canvas_width//2, canvas_height//2, image=self.photo, anchor=tk.CENTER)
        
        if not self.stop_event.is_set():
            self.window.after(10, self.update)
    
    def process_frame(self, frame):
        current_time = time.time()
        
        # Initialize the 5-second logging interval
        if self.last_log_time is None:
            self.last_log_time = current_time
        
        # Process frame for emotion detection
        frame_copy = frame.copy()
        frame_copy.flags.writeable = False
        frame_copy = cv2.cvtColor(frame_copy, cv2.COLOR_BGR2RGB)
        
        with self.mp_face_mesh.FaceMesh(
            max_num_faces=1,
            refine_landmarks=False,
            min_detection_confidence=0.5,
            min_tracking_confidence=0.5) as face_mesh:
            
            results = face_mesh.process(frame_copy)

            results = face_mesh.process(frame_copy)
            
            if results.multi_face_landmarks:
                for fl in results.multi_face_landmarks:
                    h, w, _ = frame_copy.shape
                    startX, startY, endX, endY = get_box(fl, w, h)
                    
                    # Extract face
                    cur_face = frame_copy[startY:endY, startX:endX]
                    if cur_face.size == 0:  # Skip if face not properly detected
                        continue
                    
                    try:
                        # Process with model
                        cur_face = pth_processing_np(Image.fromarray(cur_face))
                        #print(f"Processing face with shape: {cur_face.shape}")
                        features = self.session_feature.run(None, {self.feature_extractor_input_name: cur_face})[0]
                        features = np.maximum(features, 0).reshape((-1,))  # Apply ReLU activation
                        #print(f"Extracted features shape: {features.shape}")
                        # Update LSTM features
                        if len(self.lstm_features) == 0:
                            self.lstm_features = [features] * 10
                        else:
                            self.lstm_features = self.lstm_features[1:] + [features]
                        
                        lstm_f = np.array(self.lstm_features)
                        lstm_f = np.expand_dims(lstm_f, axis=0)  # Add
                        #print(f"LSTM input shape: {lstm_f.shape}")
                        output = self.session_lstm.run(None, {self.lstm_input_name: lstm_f})[0][0]
                        #print(f"LSTM output shape: {output.shape}")
                        # Get emotion label
                        cl = np.argmax(output)
                        #print(f"Predicted class index: {cl}")
                        emotion = self.DICT_EMO[cl]
                        #print(f"Detected emotion: {emotion} with confidence {output[cl]:.2f}")
                        confidence = output[cl]
                        
                        # Add to current emotions
                        self.current_emotions.append(emotion)
                    except Exception as e:
                        print(f"Error processing face: {e}")
            
        # Log emotions every 5 seconds
        if current_time - self.last_log_time >= 5 and self.current_emotions:
            self.log_emotions()
            self.last_log_time = current_time
    
    def log_emotions(self):
        if not self.current_emotions:
            return
        
        # Count occurrences of each emotion
        emotion_counts = Counter(self.current_emotions)
        dominant_emotion = emotion_counts.most_common(1)[0][0]
        
        # Check for negative emotions
        negative_emotions = [emotion for emotion in self.current_emotions if emotion in self.NEGATIVE_EMOTIONS]
        has_negative = len(negative_emotions) > 0
        
        # Create log entry
        elapsed_time = int(time.time() - self.logging_start_time)
        timestamp = f"{elapsed_time//60:02d}:{elapsed_time%60:02d}"
        
        log_entry = {
            "timestamp": timestamp,
            "dominant_emotion": dominant_emotion,
            "counts": dict(emotion_counts),
            "has_negative": has_negative
        }
        
        self.emotion_logs.append(log_entry)
        
        # Clear current emotions for next interval
        self.current_emotions = []
    
    def start_logging(self):
        self.is_running = True
        self.logging_start_time = time.time()
        self.last_log_time = None
        self.emotion_logs = []
        self.speech_logs = []
        self.current_emotions = []
        
        # Start audio processing
        self.audio_processor.start_recording()
        
        # Start stopwatch
        self.stopwatch_active = True
        self.update_stopwatch()
        
        # Update UI
        self.status_label.config(text="Logging")
        self.start_button.config(state=tk.DISABLED)
        self.stop_button.config(state=tk.NORMAL)
        self.reset_button.config(state=tk.DISABLED)
        self.analyze_button.config(state=tk.DISABLED)  # Disable analyze button during recording
        self.choose_file_button.config(state=tk.DISABLED)  # Disable file selection during recording
        
        # Clear summary
        self.summary_text.config(state=tk.NORMAL)
        self.summary_text.delete(1.0, tk.END)
        self.summary_text.insert(tk.END, "Logging emotions and voice metrics...\n")
        self.summary_text.config(state=tk.DISABLED)
    
    def stop_logging(self):
        if self.is_running:
            self.is_running = False
            self.stopwatch_active = False
            
            # Stop audio recording and process final audio
            self.audio_processor.stop_recording()
            
            # Log any remaining emotions
            if self.current_emotions:
                self.log_emotions()
            
            # Update UI
            self.status_label.config(text="Ready")
            self.start_button.config(state=tk.DISABLED)
            self.stop_button.config(state=tk.DISABLED)
            self.reset_button.config(state=tk.NORMAL)
            self.analyze_button.config(state=tk.NORMAL)  # Enable the analyze button
            self.choose_file_button.config(state=tk.NORMAL)  # Re-enable file selection
            
            # No longer automatically display summary - user must click Analyze button
            # self.display_summary()
            
            # Save video summary to text file
            self.save_video_summary()
            # Save audio summary to text file
            self.save_audio_summary()
    
    def save_video_summary(self):
        """Save the video summary to a text file."""
        try:
            with open("video_logging.txt", "w") as file:
                file.write("== Emotion Logging Summary ==\n")
                
                # Format final time
                hours, remainder = divmod(int(self.elapsed_time), 3600)
                minutes, seconds = divmod(remainder, 60)
                time_str = f"{hours:02d}:{minutes:02d}:{seconds:02d}"
                file.write(f"Total Duration: {time_str}\n")
                
                # Log 5-second intervals
                file.write("=== 5-second Intervals ===\n")
                for log in self.emotion_logs:
                    file.write(f"[{log['timestamp']}] Dominant: {log['dominant_emotion']}\n")
                    #if log['has_negative']:
                        #file.write("⚠️ Negative emotions detected!\n")
                    for emotion, count in log['counts'].items():
                        percentage = count / sum(log['counts'].values()) * 100
                        file.write(f"* {emotion}: {percentage:.1f}%\n")
                
                # Overall summary
                file.write("=== Overall Summary ===\n")
                if self.emotion_logs:
                    dominant_counts = Counter([log['dominant_emotion'] for log in self.emotion_logs])
                    most_common = dominant_counts.most_common()
                    file.write("Most frequent emotions:\n")
                    for emotion, count in most_common:
                        percentage = count / len(self.emotion_logs) * 100
                        file.write(f"* {emotion}: {percentage:.1f}%\n")
                    
                    negative_intervals = sum(1 for log in self.emotion_logs if log['has_negative'])
                    neg_percentage = negative_intervals / len(self.emotion_logs) * 100
                    #file.write(f"⚠️ Negative emotions detected in {negative_intervals} intervals ({neg_percentage:.1f}%)\n")
            # --- Write top 2 dominant emotions to video_overall_log.txt ---
            if self.emotion_logs:
                dominant_counts = Counter([log['dominant_emotion'] for log in self.emotion_logs])
                top2 = dominant_counts.most_common(2)
                with open("video_overall_log.txt", "w") as f:
                    for emotion, count in top2:
                        percentage = count / len(self.emotion_logs) * 100
                        f.write(f"{emotion}: {percentage:.1f}%\n")
        except Exception as e:
            print(f"Error saving video summary: {e}")
    
    def save_audio_summary(self):
        """Save the audio summary to a text file."""
        try:
            with open("audio_logging.txt", "w") as f:
                f.write("== voice attribute Logging Summary ==\n")
                # total duration
                hours, rem = divmod(int(self.elapsed_time), 3600)
                mins, secs = divmod(rem, 60)
                f.write(f"Total Duration: {hours:02d}:{mins:02d}:{secs:02d}\n")
                f.write("=== 5-second Intervals ===\n")
                from collections import Counter
                doms = []
                # iterate through each logged audio entry
                for idx, log in enumerate(self.speech_logs, start=1):
                    dom_emotion = max(log['emotions'], key=log['emotions'].get)
                    doms.append(dom_emotion)
                    f.write(f"[Interval {idx}] Dominant: {dom_emotion}\n\n")
                    # write each emotion and metric
                    # for k, v in {**log['emotions'], **log['metrics']}.items():
                    #     f.write(f"{k}: {v:.1f}\n")
                    # f.write("\n")
                    # Only write relevant metrics/emotions
                    for k, v in {**log['emotions'], **log['metrics']}.items():
                        if k in ["samples_processed", "volume_db"]:
                            continue  # Skip these keys
                        if k == "volume":
                            scaled_volume = v * 100
                            f.write(f"volume: {scaled_volume:.1f}\n")
                        else:
                            f.write(f"{k}: {v:.1f}\n")
                # overall summary
                # f.write("=== Overall Summary ===\n")
                # f.write("Most frequent emotions:\n")
                # counts = Counter(doms)
                # for em, cnt in counts.items():
                #     pct = cnt / len(doms) * 100 if doms else 0
                #     f.write(f"{em}: {pct:.1f}%\n")
            # --- Write top 2 dominant emotions to audio_overall_log.txt ---
            if self.speech_logs:
                doms = []
                for log in self.speech_logs:
                    dom_emotion = max(log['emotions'], key=log['emotions'].get)
                    doms.append(dom_emotion)
                from collections import Counter
                counts = Counter(doms)
                top2 = counts.most_common(2)
                with open("audio_overall_log.txt", "w") as f2:
                    for emotion, count in top2:
                        percentage = count / len(doms) * 100 if doms else 0
                        f2.write(f"{emotion}: {percentage:.1f}%\n")
        except Exception as e:
            print(f"Error saving audio summary: {e}")
    
    def reset_logging(self):
        # Reset all variables to initial state
        self.emotion_logs = []
        self.speech_logs = []
        self.current_emotions = []
        self.logging_start_time = None
        self.last_log_time = None
        self.elapsed_time = 0
        self.lstm_features = []
        
        # Overwrite the video summary file
        try:
            with open("video_logging.txt", "w") as file:
                file.write("Ready for new logging session.\n")
        except Exception as e:
            print(f"Error resetting video summary file: {e}")
        
        # Reset UI
        self.time_label.config(text="00:00:00")
        self.status_label.config(text="Ready")
        self.start_button.config(state=tk.NORMAL)
        self.stop_button.config(state=tk.DISABLED)
        self.reset_button.config(state=tk.DISABLED)
        self.analyze_button.config(state=tk.DISABLED)  # Disable analyze button
        
        # Reset voice metrics
        # for metric in ["confidence", "enthusiasm", "clarity", "speaking rate"]:
        #     self.voice_labels[metric].config(text="N/A")
        #     self.voice_labels[f"{metric}_bar"]["value"] = 0
        
        # Clear summary
        self.summary_text.config(state=tk.NORMAL)
        self.summary_text.delete(1.0, tk.END)
        self.summary_text.insert(tk.END, "Ready to start a new session.\n")
        self.summary_text.config(state=tk.DISABLED)
    
    def display_summary(self):
        def run_report_and_display():
            # Show progress dialog
            progress_win = tk.Toplevel(self.window)
            progress_win.title("Generating Report")
            progress_win.geometry("300x80")
            progress_win.transient(self.window)
            progress_win.grab_set()
            label = ttk.Label(progress_win, text="Generating report, please wait...")
            label.pack(pady=10)
            pb = ttk.Progressbar(progress_win, mode="indeterminate")
            pb.pack(fill=tk.X, padx=20, pady=5)
            pb.start(10)
            
            def worker():
                try:
                    subprocess.run(
                        ["python", "generate_report.py"],
                        check=True,
                        cwd=os.path.dirname(os.path.abspath(__file__)) if "__file__" in globals() else os.getcwd()
                    )
                except Exception as e:
                    print(f"Error running generate_report.py: {e}")
                # Schedule finish_report to run in the main thread
                self.window.after(0, finish_report)

            def finish_report():
                pb.stop()
                progress_win.destroy()
                # Display the output from presenattion_report.txt
                report_path = "presentation_feedback.txt"
                self.summary_text.config(state=tk.NORMAL)
                self.summary_text.delete(1.0, tk.END)
                try:
                    with open(report_path, "r", encoding="utf-8") as f:
                        report_content = f.read()
                        self.summary_text.insert(tk.END, report_content)
                except Exception as e:
                    self.summary_text.insert(tk.END, f"Could not load report: {e}")
                self.summary_text.config(state=tk.DISABLED)

            

            threading.Thread(target=worker, daemon=True).start()

        # Run the report generation and display in the main thread
        self.window.after(0, run_report_and_display)
        
    def on_closing(self):
        self.stop_event.set()
        if hasattr(self, 'audio_processor'):
            self.audio_processor.stop_recording()
        if self.cap and self.cap.isOpened():
            self.cap.release()
        self.window.destroy()

In [ ]:
# Run the application
root = tk.Tk()
app = EmotionLoggingApp(root, "Presentation Rehearsal Voice & Emotion Tracker")

Facial feature extraction model loaded successfully.
Audio recording started
Audio recording stopped
Error running generate_report.py: Command '['python', 'generate_report.py']' returned non-zero exit status 1.
Error running generate_report.py: Command '['python', 'generate_report.py']' returned non-zero exit status 1.
